In [1]:
from stable_baselines3 import PPO
from mkw import MarketWatchGame
from ModifiedYahooDownloader import IntervalYahooDownloader
import time
import yfinance as yf
import pandas as pd
import numpy as np
import asyncio
from datetime import datetime, time

In [2]:
model = PPO.load("./trained_models/PPO_10K_315.zip")

In [3]:
import sys
sys.path.append("./FinRL")

In [4]:
from ModifiedPreprocessor import FeatureEngineer, data_split
from TickerList import TICKERS
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from ModifiedYahooDownloader import IntervalYahooDownloader
from stable_baselines3.common.vec_env import DummyVecEnv
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

In [5]:
TICKERS

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'FCX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'JNJ',
 'JPM',
 'KO',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'RCL',
 'SUNW',
 'TRV',
 'TSLA',
 'UNH',
 'VZ']

In [6]:
def process_data(data):
    fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

    processed = fe.preprocess_data(data)
    processed = processed.copy()
    processed = processed.fillna(0)
    processed = processed.replace(np.inf,0)

    return processed

In [7]:
def setUpData(amt):

    mwg = MarketWatchGame("youweiz@nycstudents.net", "Anto1238!", "sit-2024-stock-market-challenge")

    stocks = mwg.get_shares()
    num_stock_shares = []        

    data = IntervalYahooDownloader(
        period="2d",
        interval="1m",
        ticker_list=TICKERS
    ).fetch_data()

    data = process_data(data)
    data = data[data['tic'] != "DOW"]
    print(len(data.tic.unique()))

    for symbol in data.tic.unique():
        if stocks.get(symbol):
            num_stock_shares.append(stocks[symbol])
        else:
            num_stock_shares.append(0)

    stock_dimension = len(data.tic.unique())
    state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
    print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

    env_kwargs = {
        "hmax": 100, 
        "initial_amount": amt, 
        "buy_cost_pct": [0.001] * stock_dimension, 
        "sell_cost_pct": [0.001] * stock_dimension, 
        "state_space": state_space, 
        "stock_dim": stock_dimension, 
        "tech_indicator_list": INDICATORS,
        "num_stock_shares": num_stock_shares,
        "action_space": stock_dimension, 
        "reward_scaling": 1e-4,
        "print_verbosity":5,
        "day": len(data[data.tic == TICKERS[0]]) - 1
    }

    df = data_split(data, "2024-02-06 09:30:00-05:00", "3000-12-06 09:30:00-05:00")

    trade_env =  DummyVecEnv(
        [
            lambda: StockTradingEnv(
                df=df,

                turbulence_threshold=0.1,
                initial=True,

                model_name="PPO",
                mode="trade",
                iteration=1,

                **env_kwargs
            )
        ]
    )


    return trade_env, mwg

In [8]:
def is_market_open():
    now = datetime.now()
    # Assuming the market is the NYSE, which is open from 9:30 AM to 4:00 PM Eastern Time
    market_open_time = time(9, 30)
    market_close_time = time(16, 0)
    # Check if current time is within market hours
    if market_open_time <= now.time() <= market_close_time and now.weekday() < 5:  # Monday=0, Sunday=6
        return True
    else:
        return False


In [9]:
async def main():

    actual_stocks = TICKERS.copy()
    if ("DOW" in actual_stocks):
        actual_stocks.remove("DOW")
    actual_stocks.sort()
    stocks = {}

    actionThreshold = 0.6
    portfolioPercentagePerStock = 0.03
    total_money = 250000
    wait_intervals_minutes = 2 * 60

    while True:

        try:

            if not is_market_open():
                print("Market is closed")
                await asyncio.sleep(wait_intervals_minutes)
                continue

            trades = 0
            
            trade_env, mwg = setUpData(total_money)

            total_money = mwg.balance()
            stocks = mwg.get_shares()

            print(f"Current $: {total_money}")

            trade_obs = trade_env.get_attr("state")

            action, _states = model.predict(trade_obs)

            actions_list = action.tolist()[0]

            for index, _ in enumerate(actual_stocks):

                try:

                    symbol = actual_stocks[index]
                    value = actions_list[index]
                    
                    price = mwg.get_price(symbol=symbol)

                    if abs(value) >= actionThreshold:

                        if (value > 0): # buy
                            
                            amt_to_buy = max(1, int( (total_money * portfolioPercentagePerStock * value)/price ))

                            if (price * amt_to_buy < total_money):

                                mwg.buy(
                                    symbol=symbol, 
                                    amount=amt_to_buy 
                                )

                                stocks[symbol] += amt_to_buy
                                total_money -= price * amt_to_buy

                                trades += 1

                                print(f"\t+ Bought {amt_to_buy} {symbol} for ${price * amt_to_buy}")

                            else:
                                print(f"\t! {symbol} | Insufficient funds to buy.")

                        else: # sell

                            # amt_to_sell = min(max(1, int( stocks[symbol] * value * 100 )), stocks[symbol])
                            amt_to_sell = max(1, int( stocks[symbol] / (2 * value * -1) ))

                            if (amt_to_sell <= stocks[symbol]):

                                mwg.sell(
                                    symbol=symbol, 
                                    amount=amt_to_buy 
                                )

                                stocks[symbol] -= amt_to_sell
                                total_money += price * amt_to_sell

                                trades += 1

                                print(f"\t- Sold {amt_to_sell} {symbol} for ${price * amt_to_sell}")

                            else:
                                print(f"\t! {symbol} | Insufficient funds to sell.")
                    
                except Exception as e:
                    print(f"{actual_stocks[index]} error: {e}")


            print(f"\t[Summary] Trades this interval: {trades}")
            print(f"\t[Stocks] {stocks}")

            await asyncio.sleep(wait_intervals_minutes)

        except Exception as e:
            print(f"Error {e}")


In [10]:
await main()

Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
Market is closed
